In [1]:
import pandas as pd
import numpy as np

In [2]:
scores = pd.DataFrame({
    'name':['Adam','Bob','Dave','Fred'],
    'age' :[15,16,16,15],
    'test1' :[95,81,89,None],
    'test2' :[80,82,84,88],
    'teacher' :['Ashby','Ashby','Jones','Jones']})

In [4]:
scores.head()

,name,age,test1,test2,teacher
0,Adam,15,95.0,80,Ashby
1,Bob,16,81.0,82,Ashby
2,Dave,16,89.0,84,Jones
3,Fred,15,NaN,88,Jones


In [5]:
scores.groupby('teacher').median()

,age,test1,test2
teacher,,,
Ashby,15.5,88.0,81.0
Jones,15.5,89.0,86.0


In [6]:
# This included the age column, to ignore that we can
# slice out just the test columns
scores.groupby('teacher').median()[['test1','test2']]

,test1,test2
teacher,,
Ashby,88.0,81.0
Jones,89.0,86.0


The result of calling .groupby object.The object has grouped all the rows with the same teacher together.Calling .median on the Groupby object returns a new DataFrame object that has the median score for each teacher group.You can use multiple columns to group by as well.To find the median values for every age group for each teacher,simply group teacher and age:

In [9]:
scores.groupy(['teacher', 'age']).median()

AttributeError: 'DataFrame' object has no attribute 'groupy'

In [8]:
scores.groupby(['teacher', 'age']).agg([min, max])
# When you group by multiple columns, the results 
# has a hierarchical index or multi-level index.

name       test1       test2    
              min   max   min   max   min max
teacher age                                  
Ashby   15   Adam  Adam  95.0  95.0    80  80
        16    Bob   Bob  81.0  81.0    82  82
Jones   15   Fred  Fred   NaN   NaN    88  88
        16   Dave  Dave  89.0  89.0    84  84

In [10]:
#Pivot Tables
scores.pivot_table(index='teacher',
                  values=['test1','test2'],
                  aggfunc='median')

,test1,test2
teacher,,
Ashby,88.0,81
Jones,89.0,86


In [12]:
scores.pivot_table(index=["teacher","age"], values=["test","test2"],
                   aggfunc=[len, sum], margins=True)

KeyError: 'test'

In [13]:
scores.pivot_table(index=['teacher', 'age'],
                  values=['test1', 'test2'],
                  aggfunc='median')
# if we want to aggregrate by teacher and age, we simply use 
# a list with both of them for the index parameter

test1  test2
teacher age              
Ashby   15    95.0     80
        16    81.0     82
Jones   15     NaN     88
        16    89.0     84

In [14]:
# if we want to apply multiple functions, just use a list of them.
# here, we look at the minimum and maximum test scores by teacher:
scores.pivot_table(index='teacher',
                  values=['test1', 'test'],
                  aggfunc=[min, max])

KeyError: 'test'

One additional feature of pivot tables is the ability to add summary rows.Simply by setting margins=True

In [15]:
scores.pivot_table(index='teacher',
                  values=['test1', 'test2'],
                  aggfunc='median', margins=True)

,test1,test2
teacher,,
Ashby,88.0,81
Jones,89.0,86
All,89.0,82


melting data

In [16]:
scores = pd.DataFrame({
    'name':['Adam','Bob','Dave','Fred'],
    'age' :[15,16,16,15],
    'test1' :[95,81,89,None],
    'test2' :[80,82,84,88],
    'teacher' :['Ashby','Ashby','Jones','Jones']})

In [18]:
scores.head()

,name,age,test1,test2,teacher
0,Adam,15,95.0,80,Ashby
1,Bob,16,81.0,82,Ashby
2,Dave,16,89.0,84,Jones
3,Fred,15,NaN,88,Jones


MELTING DATA
In OLAP terms, we have facts and dimension
A fact is a value that is measured and reported on e.g in a sales scenario, facts would be the number of sales of an item and the cost of the item.
A dimension is a group of values that describe the conditions of the facts i.e the store where the item was sold,the date, and the customer.
Dimensons can be sliced to dissect the data i.e if we want ot view sales by stores

In [19]:
pd.melt(scores, id_vars=['name','age'], value_vars=['test1','test2'])

,name,age,variable,value
0,Adam,15,test1,95.0
1,Bob,16,test1,81.0
2,Dave,16,test1,89.0
3,Fred,15,test1,NaN
4,Adam,15,test2,80.0
5,Bob,16,test2,82.0
6,Dave,16,test2,84.0
7,Fred,15,test2,88.0


CREATING DUMMY VARIABLES
A dummy variable is a variable that has a value 0 or 1.Also known as an indicator variable indicates whether the presence or absense of a categorical feature is found
In our case we prefer to have the age column, with a 1 or 0

In [20]:
pd.get_dummies(scores, columns=['age'], prefix='age')

,name,test1,test2,teacher,age_15,age_16
0,Adam,95.0,80,Ashby,1,0
1,Bob,81.0,82,Ashby,0,1
2,Dave,89.0,84,Jones,0,1
3,Fred,NaN,88,Jones,1,0


Undoing dummy variables
ceating them is easy, undoing them is pretty hard

In [21]:
def undummy(df, prefix, new_col_name, val_type=float):
    ''' df - dataframe with dummy columns
        prefix - prefix of dummy columns
        new_col_name - column name to replace dummy columns
        val_type - callable type for new column
    '''
    dummy_cols = [col for col in df.columns
        if col.startwith(prefix)]
    
    #map of index loaction of dummy variable to new value
    idx2val = {i:val_type(col[len(prefix):]) for i, col
              in enumerate(dummy_cols)}
    
    def get_index(vals):#idx of dummy col to use
        return list(vals).index(1)
    
    #using the dummy_cols lookup the new value by idx
    ser = df[dummy_cols].apply(
        lambda x: idx2val.get(get_index(x), None), axis=1)
    df[new_col_name] = ser
    df = df.drop(dummy_cols, axis=1)
    return df
   

In [22]:
dum = pd.get_dummies(scores, columns=['age'], prefix='age')
undummy(dum, 'age_', 'age')

AttributeError: 'str' object has no attribute 'startwith'

In [3]:
marks_df = pd.DataFrame({
    'RegNo' : [1001, 1002, 1003, 1004, 1005,
              1006,1007, 1008, 1009, 1010],
    'Name' : ['Peter', 'John', 'Mary', 'Jane', 'Alice',
             'Jack', 'Mike', 'Joy', 'Ada', 'Ben'],
    'Score' : [70, 67, 86, 74, 63, 78, 84, 80, 68, 79]
})

In [4]:
marks_df.head()

,RegNo,Name,Score
0,1001,Peter,70
1,1002,John,67
2,1003,Mary,86
3,1004,Jane,74
4,1005,Alice,63


In [6]:
# display the 1st five rows
marks_df.head(5)

,RegNo,Name,Score
0,1001,Peter,70
1,1002,John,67
2,1003,Mary,86
3,1004,Jane,74
4,1005,Alice,63


In [12]:
# get the column names
print(marks_df.columns)

Index(['RegNo', 'Name', 'Score'], dtype='object')


In [14]:
# slicing the dataframe to print the RegNo and Score
scores_df = marks_df[['RegNo','Score']]
print(scores_df)

   RegNo  Score
0   1001     70
1   1002     67
2   1003     86
3   1004     74
4   1005     63
5   1006     78
6   1007     84
7   1008     80
8   1009     68
9   1010     79


In [15]:
# dispaly the data frame in reverse order
reverse_marks_df = marks_df.iloc[::-1]
print(reverse_marks_df)

   RegNo   Name  Score
9   1010    Ben     79
8   1009    Ada     68
7   1008    Joy     80
6   1007   Mike     84
5   1006   Jack     78
4   1005  Alice     63
3   1004   Jane     74
2   1003   Mary     86
1   1002   John     67
0   1001  Peter     70


In [17]:
import random

# generate 10 random numbers between 0 and 9
numbers = [random.randint(0, 9) for i in range(10)]

# count the number of even numbers
count = 0
for num in numbers:
    if num % 2 == 0:
        count += 1
        print(num)
        
print(f'Total even numbers: {count}')

8
2
0
0
0
Total even numbers: 5


In [18]:
letters = ['a', 'b', 'o', 'c', 'p']
print(letters)

['a', 'b', 'o', 'c', 'p']


In [19]:
letters[1]

'b'

In [20]:
letters[len(letters)-2]

'c'

In [21]:
letters[-1]

'p'

In [23]:
this creates a tuple
food_prices = ({
    'Items' : ['Sugar', 'Salt', 'Milk', 'Bread'],
    'Price' : [120, 20, 70, 50]
    })
print(food_prices)

{'Items': ['Sugar', 'Salt', 'Milk', 'Bread'], 'Price': [120, 20, 70, 50]}


In [25]:
#for a dictionary
food_prices ={
    'Sugar' : 120,
    'Salt': 20,
    'Milk': 70,
    'Bread': 50
}
print(food_prices)

{'Sugar': 120, 'Salt': 20, 'Milk': 70, 'Bread': 50}


In [27]:
fruits_prices = {
    'Apples' : 100,
    'Bananas' : 150,
    'Oranges' : 50,
    'Mangoes' : 80
}

print (fruits_prices)

{'Apples': 100, 'Bananas': 150, 'Oranges': 50, 'Mangoes': 80}


In [28]:
# merging the two dictionaries we have
food_prices.update(fruits_prices)
print(food_prices)

{'Sugar': 120, 'Salt': 20, 'Milk': 70, 'Bread': 50, 'Apples': 100, 'Bananas': 150, 'Oranges': 50, 'Mangoes': 80}


In [29]:
# replace price of mangoes
food_prices['mangoes'] = 65
print(food_prices)

{'Sugar': 120, 'Salt': 20, 'Milk': 70, 'Bread': 50, 'Apples': 100, 'Bananas': 150, 'Oranges': 50, 'Mangoes': 80, 'mangoes': 65}


In [30]:
# delete salt
del food_prices['Salt']
print(food_prices)

{'Sugar': 120, 'Milk': 70, 'Bread': 50, 'Apples': 100, 'Bananas': 150, 'Oranges': 50, 'Mangoes': 80, 'mangoes': 65}
